# Model Context Protocol
This notebook provides an overview of MCP client.

Server code runs in .py files.

Langchain provide a wrapper library for MCP.  Install with: 
- uv add "mcp[cli]"

The library creates a unified object that contains connections to mulitple MCP servers.

Langchain also offer the ability to inject arguments into tool calls.  This allows the developer to remove arguments that the LLM does not have to (or should not) see.  The is an important feature for both context management and security.  

In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient

MCP servers to connect to can be listed in the argument for MultiServerMCPCLient construction.

The example below connects to the MCP server in 'get-time-server.py'.  Start the server using uv run (or python) and leave it running on the command line.  The server runs in streamable_http transport mode.



In [44]:
client = MultiServerMCPClient(
    {
        "math": {
            "transport": "streamable_http",
            "url": "http://localhost:8000/mcp",
        }
    }
)

Note that while MultiServerMCPClient can also connect using 'stdio' transport mode, it does not appear to work in Jupyter.  An example is shown below, and will run in python outside of Jupyter.

In [ ]:
stdio_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uv",
            "args": ["run","C:/Users/Andrew/Documents/dkit-projects/agentic-labs/MCP/get-time.py"],
        }
    }
)


In [50]:
# tools list from time server

tools = await client.get_tools()
print(tools)

[StructuredTool(name='get_current_time', description='Get the current time in ISO 8601 format.', args_schema={'properties': {}, 'title': 'get_current_timeArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000001CA0E7EA700>)]


# Record Shop with MCP
The example below ca remote server for tools.
Since MCP tool call is async, all invoke calls must be async as well.

llm.invoke(...) becomes:  await llm.ainvoke(...)

In [48]:

from langchain_google_genai import ChatGoogleGenerativeAI
# import HumanMessages for the LLM invocation
from langchain.messages import HumanMessage, SystemMessage
# import checkpointer
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig

from langgraph.graph import MessagesState

from langchain.messages import AnyMessage, RemoveMessage

from langgraph.prebuilt import ToolNode, InjectedState
from langchain.tools import tool, ToolRuntime
from langchain_community.tools.file_management.read import ReadFileTool

from langgraph.graph import StateGraph, END, START
from langgraph.types import interrupt, Command

from pydantic import BaseModel, Field
from typing import Optional

import os
from dotenv import load_dotenv
load_dotenv()

mcp_client = MultiServerMCPClient(
    {
        "music_catalog_srv": {
            "transport": "streamable_http",
            "url": "http://localhost:8001/mcp",
        }
    }
)

tools = await mcp_client.get_tools()

llm = ChatGoogleGenerativeAI(
    model=os.getenv("GOOGLE_API_MODEL"),
    temperature=0)

class RecordShopState(MessagesState):
    email: str
    success: bool
    count: int = 0

async def action(state: RecordShopState) -> dict:
    result = await llm.bind_tools(tools).ainvoke(state['messages'])
    return {"messages": result}

graph = StateGraph(RecordShopState)
graph.add_node("action", action)
graph.add_node("tools", ToolNode(tools))

graph.add_edge(START, "action")
# conditional edge - if tool_calls == [] for to END else to "tools"
graph.add_conditional_edges("action", 
    lambda state: state['messages'][-1].tool_calls == [],
    path_map={True: END, False: "tools"}
    )
graph.add_edge("tools", "action")

compiled_graph = graph.compile()
    

system_prompt_react = """
You are a helpful assistant in a record shop that sells albums.  
You are answering queries from emails about what albums are available or making recommendations.
Answer the queries in a friendly and informative way.  If the email includes the customer name please use it. Sign your emails in the name of Andrew.
Think step by step and call the tools as needed to get the information required to answer the query.

"""
prompt_react = """


The query is:
{query}
"""
import pandas as pd

df = pd.read_csv("emails.csv")
for index, row in df.iterrows():
    email = row['email']
    run = row['run']
    if run == "yes":
        messages = [SystemMessage(content=system_prompt_react),
                    HumanMessage(content=prompt_react.format(query=email))]
        response = await compiled_graph.ainvoke(input=RecordShopState(messages=messages, success=True, count=0, email=email), config=RunnableConfig(recursion_limit=100))
        # response = main_react(email)
        df.at[index, 'response'] = response['messages'  ][-1].content
        print(response['messages'][-1].content)
# response['messages'][-1]

Hi Tom,

Yes, we have several David Bowie albums available! Here's what we have in stock:

*   **The Rise and Fall of Ziggy Stardust and the Spiders From Mars** (1972) - Rock, Classic Rock, Glam
*   **Hunky Dory** (1971) - Rock, Classic Rock, Glam
*   **Low** (1977) - Electronic, Rock, Art Rock, Ambient, Experimental
*   **Aladdin Sane** (1973) - Rock, Glam
*   **Station to Station** (1976) - Rock, Funk / Soul, Classic Rock, Soul, Funk, Art Rock

Let me know if you'd like more information on any of these or if you're looking for something else!

Cheers,
Andrew


AIMessage(content="Hi Tom,\n\nYes, we have several David Bowie albums available! Here's what we have in stock:\n\n*   **The Rise and Fall of Ziggy Stardust and the Spiders From Mars** (1972) - Rock, Classic Rock, Glam\n*   **Hunky Dory** (1971) - Rock, Classic Rock, Glam\n*   **Low** (1977) - Electronic, Rock, Art Rock, Ambient, Experimental\n*   **Aladdin Sane** (1973) - Rock, Glam\n*   **Station to Station** (1976) - Rock, Funk / Soul, Classic Rock, Soul, Funk, Art Rock\n\nLet me know if you'd like more information on any of these or if you're looking for something else!\n\nCheers,\nAndrew", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--e70637b8-5e15-43a9-ae89-4f033ed64f9e-0', usage_metadata={'input_tokens': 700, 'output_tokens': 180, 'total_tokens': 880, 'input_token_details'

## Connecting to Tavily Remote MCP server

We can connect to vendor hosted MCP servers using the remote HTTP URL.

Below is an example using the Tavily hosted MCP server.  We can get the list of tools and execute queries all through the remote Cloud environment.  Note, you will need a Tavily API key to perform this connection.

In [52]:
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [ ]:
tavily_server = MultiServerMCPClient(
    {
        "tavily_srv": {
            "transport": "streamable_http",
            "url": f"https://mcp.tavily.com/mcp/?tavilyApiKey={tavily_api_key}",
        }
    })
    
    

In [ ]:
tools = await tavily_server.get_tools()
print(tools)

In [56]:
# import MessagesState
from langgraph.graph import MessagesState

async def action(state: MessagesState) -> MessagesState:
    result = await llm.ainvoke(state['messages'])
    return {"messages": result}

graph = StateGraph(MessagesState)
graph.add_node("action", action)
graph.add_node("tools", ToolNode(tools))
graph.add_edge(START, "action")
graph.add_conditional_edges("action", 
    lambda state: state['messages'][-1].tool_calls == [],
    path_map={True: END, False: "tools"}
    )
graph.add_edge("tools", "action")
compiled_graph = graph.compile()

messages = [SystemMessage(content="You are a helpful assistant."),
            HumanMessage(content="Search for web pages on MCP")]
response = await compiled_graph.ainvoke(input=MessagesState(messages=messages))
print(response['messages'][-1].content)



"MCP" is an acronym that can stand for several different things, depending on the context. To help you find what you're looking for, here are some of the most common meanings and related web pages:

1.  **Microsoft Certified Professional (MCP):** This is a very common meaning, referring to individuals who have passed certification exams for Microsoft products and technologies.
    *   You'll find information on Microsoft's official learning and certification pages, as well as forums and training providers.
    *   *Search terms:* "Microsoft Certified Professional," "MCP certification," "Microsoft exams"

2.  **Monocalcium Phosphate (MCP):** A chemical compound often used as a leavening agent in baking (e.g., in baking powder) or as a feed additive for livestock.
    *   You'd find information on food science sites, chemical suppliers, and agricultural resources.
    *   *Search terms:* "Monocalcium Phosphate," "MCP food additive," "MCP chemical"

3.  **Master Control Program (MCP):** A

## Using MCP Injection

This is value for allowing arguments to be sent to tool without needing to go to model.

However, appears to have issues in environment or release at present

In [ ]:
from dataclasses import dataclass
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.interceptors import MCPToolCallRequest


# Notes on Running MCP Servers

Example servers in the MCP folder on Moodle should be run on the command line, prefferably with UV. For instance:
>  uv run get-time-svr.py

MCP servers can be tested through a web front end by using the MCP Inspector tool, found at:
- https://github.com/modelcontextprotocol/inspector